In [ ]:
!pip install --upgrade pip
!pip install -r requirements.txt --user

In [ ]:
import time
from datetime import datetime

import boto3
from sagemaker import get_execution_role

# from sagemaker.experiments.experiment import Experiment

sm = boto3.Session().client("sagemaker")
role = get_execution_role()
print(role)

In [ ]:
def get_regions():
    ec2 = boto3.client("ec2")
    regions = ec2.describe_regions()
    return [r["RegionName"] for r in regions["Regions"]]

In [ ]:
def print_delete_result(_type, _name, deleted=False):
    if deleted is True:
        print(f"\n{_type} {_name} deleted\n")
    else:
        print(f"\nUnable to delete {_type} {_name}. It may not exist.")

In [ ]:
def search_projects(_project):
    projects = sm.list_projects(NameContains=_project)

    if "NextToken" in projects:
        next_token = projects["NextToken"]
        print(f"{next_token}\n")

    print("ProjectNames:")
    for project in projects["ProjectSummaryList"]:
        project_name = project["ProjectName"]
        print(f"\n{project_name}")

In [ ]:
def cleanup_experiment(_experiment_name):
    try:
        trials = sm.list_trials(ExperimentName=_experiment_name)["TrialSummaries"]
        print("TrialNames:")
        for trial in trials:
            trial_name = trial["TrialName"]
            print(f"\n{trial_name}")

            components_in_trial = sm.list_trial_components(TrialName=trial_name)
            print("\tTrialComponentNames:")
            for component in components_in_trial["TrialComponentSummaries"]:
                component_name = component["TrialComponentName"]
                print(f"\t{component_name}")
                sm.disassociate_trial_component(TrialComponentName=component_name, TrialName=trial_name)
                try:
                    # comment out to keep trial components
                    sm.delete_trial_component(TrialComponentName=component_name)
                    print_delete_result("Trial Component", component_name, True)
                except:
                    # component is associated with another trial
                    continue
                # to prevent throttling
                time.sleep(0.5)
            sm.delete_trial(TrialName=trial_name)
            print_delete_result("Trial", trial_name, True)
    except:
        print_delete_result("Trial", "", False)     
        
    try:    
        sm.delete_experiment(ExperimentName=_experiment_name)
        print_delete_result("Experiment", _experiment_name, True)
    except:
        print_delete_result("Experiment", _experiment_name, False)

In [ ]:
def cleanup_model(_model_name):
    models = sm.list_models(NameContains=_model_name)
    model_bias_job_definitions = sm.list_model_bias_job_definitions(NameContains=_model_name)
    model_explainability_job_definitions = sm.list_model_explainability_job_definitions(
        NameContains=_model_name
    )
    # model_metadata = sm.list_model_metadata(NameContains=_model_name)
    model_package_groups = sm.list_model_package_groups(NameContains=_model_name)
    model_quality_job_definitions = sm.list_model_quality_job_definitions(NameContains=_model_name)

    if "NextToken" in models:
        next_token = models["NextToken"]
        print(f"\n{next_token}")

    print("JobDefinitions:")
    for model in model_bias_job_definitions["JobDefinitionSummaries"]:
        model_name = model["EndpointName"]
        print(f"\n{model_name}")

    print("JobDefinitions:")
    for model in model_explainability_job_definitions["JobDefinitionSummaries"]:
        model_name = model["EndpointName"]
        print(f"\n{model_name}")

    # print("ModelNames:")
    # for model in model_metadata["Models"]:
    #     model_name = model["ModelName"]
    #     print(f"\n{model_name}")

    print("ModelPackageGroupNames:")
    for group in model_package_groups["ModelPackageGroupSummaryList"]:
        group_name = group["ModelPackageGroupName"]
        print(f"\n{group_name}")

        model_packages = sm.list_model_packages(ModelPackageGroupName=group_name)
        print("ModelPackageNames:")
        for package in model_packages["ModelPackageSummaryList"]:
            print("\nPackage data")
            print(f"{package}")
            if "ModelPackageName" in package:
                package_name = package["ModelPackageName"]
                print(f"\n{package_name}")
                # sm.delete_model_package(ModelPackageName=package_name)
            if "ModelPackageArn" in package:
                package_arn = package["ModelPackageArn"]
                print(f"\n{package_arn}")
                sm.delete_model_package(ModelPackageName=package_arn)
                print_delete_result("Model Package", package_arn, True)

        sm.delete_model_package_group(ModelPackageGroupName=group_name)
        print_delete_result("Model Package Group", group_name, True)

    print("JobDefinitions:")
    for model in model_quality_job_definitions["JobDefinitionSummaries"]:
        model_name = model["EndpointName"]
        print(f"\n{model_name}")

    print("ModelNames:")
    for model in models["Models"]:
        model_name = model["ModelName"]
        sm.delete_model(ModelName=model_name)
        print_delete_result("Model", model_name, True)

In [ ]:
def cleanup_endpoint_dont_use(_endpoint_name):
    endpoints = sm.list_endpoints(NameContains=_endpoint_name)

    if "NextToken" in endpoints:
        next_token = endpoints["NextToken"]
        print(f"\n{next_token}")

    print("EndpointNames:")
    for endpoint in endpoints["Endpoints"]:
        endpoint_name = endpoint["EndpointName"]
        cleanup_endpoint(endpoint_name)

    #     components_in_trial = sm.list_trial_components(TrialName=trial_name)
    #     print("\tTrialComponentNames:")
    #     for component in components_in_trial["TrialComponentSummaries"]:
    #         component_name = component["TrialComponentName"]
    #         print(f"\t{component_name}")
    #         sm.disassociate_trial_component(TrialComponentName=component_name, TrialName=trial_name)
    #         try:
    #             # comment out to keep trial components
    #             sm.delete_trial_component(TrialComponentName=component_name)
    #         except:
    #             # component is associated with another trial
    #             continue
    #         # to prevent throttling
    #         time.sleep(0.5)
    #     sm.delete_trial(TrialName=trial_name)
    # sm.delete_experiment(ExperimentName=_experiment_name)
    # print(f"\nExperiment {_experiment_name} deleted")

In [ ]:
def cleanup_monitoring_schedules(_monitoring_schedule_name):
    try:
        sm.delete_monitoring_schedule(MonitoringScheduleName=_monitoring_schedule_name)
        print_delete_result("Monitoring Schedule", _monitoring_schedule_name, True)
    except:
        print_delete_result("Monitoring Schedule", _monitoring_schedule_name, False)

In [ ]:
def list_endpoints():    
    endpoints = sm.list_endpoints()
    for endpoint in endpoints["Endpoints"]:
        endpoint_name = endpoint["EndpointName"]
        print(endpoint_name)

In [ ]:
def cleanup_endpoint(_endpoint_name):
    try:
        sm.delete_endpoint(EndpointName=_endpoint_name)
        print_delete_result("Endpoint", _endpoint_name, True)
    except:
        print_delete_result("Endpoint", _endpoint_name, False)

In [ ]:
def list_endpoint_configs():    
    endpoint_configs = sm.list_endpoint_configs()
    for endpoint_config in endpoint_configs["EndpointConfigs"]:
        endpoint_config_name = endpoint_config["EndpointConfigName"]
        print(endpoint_config_name)

In [ ]:
def cleanup_endpoint_configs(_endpoint_configs_name):
    try:
        sm.delete_endpoint_config(EndpointConfigName=_endpoint_configs_name)
        print_delete_result("Endpoint Config", _endpoint_configs_name, True)
    except:
        print_delete_result("Endpoint Config", _endpoint_configs_name, False)

In [ ]:
def list_hyper_parameter_tuning_jobs():
    hyper_parameter_tuning_jobs = sm.list_hyper_parameter_tuning_jobs()
    # print(hyper_parameter_tuning_jobs)
    for hyper_parameter_tuning_job in hyper_parameter_tuning_jobs["HyperParameterTuningJobSummaries"]:
        hyper_parameter_tuning_job_name = hyper_parameter_tuning_job["HyperParameterTuningJobName"]
        print(hyper_parameter_tuning_job_name)

In [ ]:
# def delete_hyper_parameter_tuning_job(_hyper_parameter_tuning_job_name):
#     try:
#         sm.delete_hyper_parameter_tuning_job(HyperParameterTuningJobName=_hyper_parameter_tuning_job_name)
#         print_delete_result("HyperParameterTuningJob", _hyper_parameter_tuning_job_name, True)
#     except:
#         print_delete_result("HyperParameterTuningJob", _hyper_parameter_tuning_job_name, False)    

In [ ]:
def cleanup_project(project_name):
    try:
        sm.delete_project(ProjectName=project_name)
        print_delete_result("Project", project_name, True)
    except:
        print_delete_result("Project", project_name, False)

In [ ]:
def cleanup_pipeline(_pipeline_name):
    pipelines = sm.list_pipelines(PipelineNamePrefix=_pipeline_name)
    print(f"{pipelines}\n")

    if "NextToken" in pipelines:
        next_token = pipelines["NextToken"]
        print(f"{next_token}\n")

    print("PipelineSummaries:")
    for pipeline in pipelines["PipelineSummaries"]:
        pipeline_name = pipeline["PipelineName"]
        pipeline_arn = pipeline["PipelineArn"]
        print(f"{pipeline_name}")
        print(f"{pipeline_arn}")
        sm.delete_pipeline(PipelineName=pipeline_name)
        print_delete_result("Pipeline", pipeline_name, True)

In [ ]:
def list_notebooks(_notebook_name=""):
    notebooks = sm.list_notebook_instances(MaxResults=100)
    print(f"{notebooks}\n")

    if "NextToken" in notebooks:
        next_token = notebooks["NextToken"]
        print(f"{next_token}\n")

    print("NotebookInstances:")
    for notebook in notebooks["NotebookInstances"]:
        notebook_name = notebook["NotebookInstanceName"]
        notebook_arn = notebook["NotebookInstanceArn"]
        print(f"{notebook_name}")
        print(f"{notebook_arn}")
        # sm.delete_pipeline(PipelineName=pipeline_name)
        # print(f"{notebook_name} stopped")

In [ ]:
def stop_notebooks(_notebook_name=""):
    notebooks = sm.list_notebook_instances(MaxResults=100)
    # print(f"{notebooks}\n")

    if "NextToken" in notebooks:
        next_token = notebooks["NextToken"]
        print(f"{next_token}\n")

    print("NotebookInstances:")
    for notebook in notebooks["NotebookInstances"]:
        notebook_name = notebook["NotebookInstanceName"]
        notebook_arn = notebook["NotebookInstanceArn"]
        notebook_status = notebook["NotebookInstanceStatus"]

        print(f"{notebook_name}")
        print(f"{notebook_arn}")
        print(f"{notebook_status}")
        if notebook_status == "InService":
            sm.stop_notebook_instance(NotebookInstanceName=notebook_name)
            print(f"{notebook_name} stopped")

In [ ]:
def list_domains(_domain_name=""):
    domains = sm.list_domains()
    print(f"\n{domains}\n")

    if "NextToken" in domains:
        next_token = domains["NextToken"]
        print(f"\n{next_token}")

    print("Domains:")
    for domain in domains["Domains"]:
        domain_name = domain["DomainName"]
        domain_id = domain["DomainId"]
        domain_arn = domain["DomainArn"]
        print(f"{domain_name}")
        print(f"{domain_id}")
        print(f"{domain_arn}")
    # sm.delete_pipeline(PipelineName=pipeline_name)
    # print(f"{notebook_name} stopped")

In [ ]:
def list_training_jobs(_jobname):
    print("list_training_jobs\n")
    search_params = {
        "MaxResults": 10,
        "Resource": "TrainingJob",
        "SearchExpression": {
            "Filters": [{"Name": "TrainingJobName", "Operator": "Contains", "Value": _jobname}]
            # "Filters": [{"Name": "Tags.sm-jumpstart-id", "Operator": "Equals", "Value": _project}]
        }
        # "SortBy": "Metrics.train:binary_classification_accuracy",
        # "SortOrder": "Descending",
    }
    results = sm.search(**search_params)
    # print(results)
    return results

In [ ]:
def cleanup_training_jobs(_jobname):
    print("cleanup_training_jobs\n")
    try:
        results = list_training_jobs(_jobname)
        # print(training_jobs)
        # print(training_jobs['Results'])
        for result in results['Results']:
            # print(result['TrainingJob'])
            training_job = result['TrainingJob']
            training_job_name = training_job['TrainingJobName']
            print(f"training_job_name: {training_job_name}")
            sm.stop_training_job(TrainingJobName=training_job_name)
            print_delete_result("TrainingJob", training_job_name, True)
    except:
        print_delete_result("TrainingJob", _jobname, False)

In [ ]:
def list_experiments():
    print("list_experiments\n")
    experiments = sm.list_experiments(
        # CreatedAfter=datetime(2015, 1, 1),
        # CreatedBefore=datetime(2015, 1, 1),
        SortBy="CreationTime",  # "Name" | "CreationTime"
        SortOrder="Descending",  # "Ascending" | "Descending"
        # NextToken="string",
        # MaxResults=123,
    )

    print(f"\n{experiments}")

    if "NextToken" in experiments:
        next_token = experiments["NextToken"]
        print(f"\n{next_token}")

    print("Domains:")
    for experiment in experiments["ExperimentSummaries"]:
        experiment_name = experiment["ExperimentName"]
        # domain_id = domain["DomainId"]
        # domain_arn = domain["DomainArn"]
        print(f"{experiment_name}")
        # print(f"{domain_id}")
        # print(f"{domain_arn}")

In [ ]:
def list_models():
    print("list_models\n")
    models = sm.list_models(
        # CreationTimeAfter=datetime(2015, 1, 1),
        CreationTimeBefore=datetime(2023, 1, 1),
        SortBy="CreationTime",  # "Name" | "CreationTime"
        SortOrder="Descending",  # "Ascending" | "Descending"
        # NextToken="string",
        # MaxResults=123,
    )

    print(f"\n{models}")

In [ ]:
def get_experiments(_next_token=""):
    print("get_experiments\n")

    if len(_next_token) > 0:
        return sm.list_experiments(
            # CreatedAfter=datetime(2015, 1, 1),
            CreatedBefore=datetime(2023, 1, 1),
            SortBy="CreationTime",  # "Name" | "CreationTime"
            SortOrder="Descending",  # "Ascending" | "Descending"
            NextToken=_next_token,
            MaxResults=10,
        )
    else:
        return sm.list_experiments(
            # CreatedAfter=datetime(2015, 1, 1),
            CreatedBefore=datetime(2023, 1, 1),
            SortBy="CreationTime",  # "Name" | "CreationTime"
            SortOrder="Descending",  # "Ascending" | "Descending"
            MaxResults=10,
        )

In [ ]:
def get_models(_next_token=""):
    print("get_models\n")

    if len(_next_token) > 0:
        return sm.list_models(
            # CreationTimeAfter=datetime(2015, 1, 1),
            CreationTimeBefore=datetime(2023, 1, 1),
            SortBy="CreationTime",  # "Name" | "CreationTime"
            SortOrder="Descending",  # "Ascending" | "Descending"
            NextToken=_next_token,
            MaxResults=10,
        )
    else:
        return sm.list_models(
            # CreationTimeAfter=datetime(2015, 1, 1),
            CreationTimeBefore=datetime(2023, 1, 1),
            SortBy="CreationTime",  # "Name" | "CreationTime"
            SortOrder="Descending",  # "Ascending" | "Descending"
            MaxResults=10,
        )

In [ ]:
def cleanup_model_packages(mpg_name):

    print("cleanup_model_packages\n")
    try:
        mpg = sm.list_model_packages(
            ModelPackageGroupName=mpg_name,
        )

        # Delete model packages if Group not empty
        model_packages = mpg.get('ModelPackageSummaryList')
        if model_packages:
            for mp in model_packages:
                sm.delete_model_package(
                    ModelPackageName=mp['ModelPackageArn']
                )
                time.sleep(1)

        # Delete model package group
        sm.delete_model_package_group(
            ModelPackageGroupName=mpg_name
        )
        print_delete_result("ModelPackages", mpg_name, True)
    except:
        print_delete_result("ModelPackages", mpg_name, False)    

In [ ]:
def list_user_profiles(_domain_id, _username=""):
    print("list_user_profiles\n")
    
    if len(_username) > 0:
         user_profiles = sm.list_user_profiles(
            DomainIdEquals=_domain_id,
            UserProfileNameContains=_username
        )
    else:
        user_profiles = sm.list_user_profiles(
            DomainIdEquals=_domain_id
        )
    # print(f"\n{user_profiles}")
    for user_profile in user_profiles["UserProfiles"]:
        user_profile_name = user_profile["UserProfileName"]
        print(f"{user_profile_name}")

In [ ]:
def delete_user_profile(_domain_id, _username):
    print("delete_user_profile\n")
    
    try:
        sm.delete_user_profile(
            DomainId=_domain_id,
            UserProfileName=_username
        )
    
        print_delete_result("UserProfile", _username, True)
    except:
        print_delete_result("UserProfile", _username, False) 

In [ ]:
def list_sagemaker_apps(_domain_id, app_name=""):
    print("list_sagemaker_apps\n")
    
    if len(app_name) > 0:
         sagemaker_apps = sm.list_apps(
            DomainIdEquals=_domain_id,
            UserProfileNameContains=_app
        )
    else:
        sagemaker_apps = sm.list_apps(
            DomainIdEquals=_domain_id
        )
    # print(f"\n{sagemaker_apps}")
    for sagemaker_app in sagemaker_apps["Apps"]:
        user_profile_name = sagemaker_app["UserProfileName"]
        app_type = sagemaker_app["AppType"]
        app_name = sagemaker_app["AppName"]
        app_status = sagemaker_app["Status"]
        
        print(f"{user_profile_name}")
        print(f"{app_type}")
        print(f"{app_name}")
        print(f"{app_status}\n")

In [ ]:
def delete_sagemaker_app(_domain_id, _username, _app_type, _app_name):
    print("delete_sagemaker_app\n")
    
    try:
        sm.delete_app(
            DomainId=_domain_id,
            UserProfileName=_username,
            AppType=_app_type,
            AppName=_app_name
        )
    
        print_delete_result("Sagemaker App", _app_name, True)
    except:
        print_delete_result("Sagemaker App", _app_name, False) 

In [ ]:
def cleanup_sagemaker_apps(_domain_id, delete_all=False):
    print("cleanup_sagemaker_apps\n")
    
    sagemaker_apps = sm.list_apps(
        DomainIdEquals=_domain_id
    )
    
    # print(f"\n{sagemaker_apps}")
    for sagemaker_app in sagemaker_apps["Apps"]:
        user_profile_name = sagemaker_app["UserProfileName"]
        app_type = sagemaker_app["AppType"]
        app_name = sagemaker_app["AppName"]
        app_status = sagemaker_app["Status"]
        
        print(f"{user_profile_name}")
        print(f"{app_type}")
        print(f"{app_name}")
        print(f"{app_status}\n") 
        
        if delete_all==True and app_status=="InService":
            delete_sagemaker_app(_domain_id, user_profile_name, app_type, app_name)
        else:
            if app_type!="JupyterServer" and app_status=="InService":
                delete_sagemaker_app(_domain_id, user_profile_name, app_type, app_name)

In [ ]:
# cleanup_experiment("pipelines-hep11qrercy0-Tra-ProfilerReport-1691645888-76d4c6e5-aws-processing-job")

In [ ]:
# search_projects("test2")

In [ ]:
# cleanup_model("pytorch-ic-mobilenet-v2-20230630-053753")

In [ ]:
# cleanup_project("test1-1-3")

In [ ]:
# list_endpoints()

In [ ]:
# cleanup_endpoint("fraud-detect-demo-xgboost-post-smote-endpoint")

In [ ]:
# cleanup_pipeline("test2-1-3-test2-1-3")

In [ ]:
# list_notebooks()

In [ ]:
# stop_notebooks()

In [ ]:
# list_domains()

In [ ]:
# list_training_jobs("pipelines-hep11qrercy0-Tra-ProfilerReport-1691645888-76d4c6e5-aws-processing-job")

In [ ]:
# cleanup_training_jobs("pipelines-yovlk5yf2sph-CustomerChurnTrain-bFwU1TUKQj")

In [ ]:
# list_models()

In [ ]:
# models = get_models()
# print(f"\n{models}")
# max_loops = 3
# current_loop = 0


# while True:
#     next_token = ""
#     current_loop = current_loop + 1

#     if "NextToken" in models:
#         next_token = models["NextToken"]
#         print(f"\n{next_token}")

#     print("Models:")
#     for model in models["Models"]:
#         model_name = model["ModelName"]
#         # domain_id = domain["DomainId"]
#         # domain_arn = domain["DomainArn"]
#         print(f"{model_name}")
#         # print(f"{domain_id}")
#         # print(f"{domain_arn}")

#         print(f"{model_name[:22]}")

#         if model_name[:22] == "sagemaker-xgboost-2021":
#             cleanup_model(model_name)
#             print("")

#     if len(next_token) == 0:
#         break

#     if current_loop == 3:
#         break

#     models = get_models(next_token)

In [ ]:
# list_endpoint_configs()

In [ ]:
# cleanup_endpoint_configs("imageidsbx-xgb-model-27-06-2023-04-49-48")

In [ ]:
# list_hyper_parameter_tuning_jobs()

In [ ]:
# cleanup_model_packages("test2-1-3-test2-1-3")

In [ ]:
# list_user_profiles("d-exyuk9ar3nj6")

In [ ]:
# delete_user_profile("d-2orjhqoncime", "sonali-suhane-fmgl-com-au-f33")

In [ ]:
# list_sagemaker_apps("d-exyuk9ar3nj6", True)

In [ ]:
# delete_sagemaker_app("d-exyuk9ar3nj6", "jason-pascoe-fmgl-com-au-cd6", "KernelGateway", "sagemaker-data-science-ml-m5-large-7b47fa541721b8624e59dbef0f5c")

In [ ]:
# cleanup_sagemaker_apps("d-exyuk9ar3nj6")

In [ ]:
cleanup_sagemaker_apps("d-exyuk9ar3nj6", True) # sandbox-AdvancedAnalytics
cleanup_sagemaker_apps("d-kdxm9mz64f34", True) # AdvancedAnalytics-Development
cleanup_sagemaker_apps("d-hi9bvtcoaodl", True) # dataanalytics-prod
cleanup_sagemaker_apps("d-2tbfyrzxw1gd", True) # integrated-operations-nonprod
cleanup_sagemaker_apps("d-fzitqoism1r5", True) # health-and-safety-prod
cleanup_sagemaker_apps("d-3vgzytoppeft", True) # ffi-data-analytics-prod
cleanup_sagemaker_apps("d-ydbl7vwzbskn", True) # AdvancedAnalytics-PreProd
cleanup_sagemaker_apps("d-gwwwk6x5t52u", True) # AdvancedAnalytics-Prod
# cleanup_sagemaker_apps("d-") # mlops-sandbox